In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import StandardScaler

In [3]:
df = pd.read_csv("../data/location_rev.csv")
df.head()

,revenue,num_competitors,median_income,num_loyalty_members,population_density,location_age
0,42247.80,3.0,30527.57,1407.0,3302.0,12.0
1,38628.37,3.0,30185.49,1025.0,4422.0,11.0
2,39715.16,1.0,32182.24,1498.0,3260.0,12.0
3,35593.30,5.0,29728.65,2340.0,4325.0,10.0
4,35128.18,4.0,30691.17,847.0,3774.0,11.0


In [4]:
df.describe()

,revenue,num_competitors,median_income,num_loyalty_members,population_density,location_age
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,38433.469330,3.397000,31360.668500,1597.200000,3351.199000,11.041000
std,11665.825242,1.016082,3943.278358,496.874663,975.664263,3.034885
min,5000.000000,0.000000,20000.000000,0.000000,0.000000,0.000000
25%,30277.897500,3.000000,28792.592500,1253.000000,2689.250000,9.000000
50%,38323.095000,3.000000,31134.555000,1605.000000,3353.000000,11.000000
75%,45894.670000,4.000000,34050.992500,1925.250000,4017.000000,13.000000
max,79342.070000,7.000000,43676.900000,3280.000000,6489.000000,24.000000


In [5]:
X = df[["num_competitors",
        "median_income",
        "population_density",
        "num_loyalty_members",
        "location_age"]]
y=df["revenue"]

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [14]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
import time

models = [LinearRegression(), KNeighborsRegressor(), DecisionTreeRegressor(), RandomForestRegressor(), SVR()]
scores = []
train_times = []
names= []
for model in models:
    start = time.time()
    # cross_val_score trả về mảng, lấy trung bình
    score = cross_val_score(estimator=model, X=X_train, y=y_train, scoring="r2", cv=5).mean()
    end = time.time()
    scores.append(score)
    train_times.append(end - start)
    names.append(model.__class__.__name__)
    
df = pd.DataFrame(scores, columns=["Scores"], index=range(len(models)))
df.insert(1, "Time", pd.Series(train_times))
df.insert(0, "Model", pd.Series(names))
df.head(10)

,Model,Scores,Time
0,LinearRegression,0.805874,0.009999
1,KNeighborsRegressor,0.681357,0.014001
2,DecisionTreeRegressor,0.483278,0.019998
3,RandomForestRegressor,0.723669,1.031928
4,SVR,-0.011709,0.051997


In [11]:
from sklearn.model_selection import GridSearchCV

grid_params = {
    "n_estimators": [50,100,200],
    "max_depth": [10,20,30, None],
    "min_samples_split": [2,5,10],
    "min_samples_leaf": [1,2,4],
    "max_features": ["auto","sqrt","log2"]
}
model = RandomForestRegressor(n_jobs=1)
model_grid = GridSearchCV(model, grid_params, cv=5)
model_grid.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestRegressor(n_jobs=1),
             param_grid={'max_depth': [10, 20, 30, None],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [50, 100, 200]})

In [12]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print(r2, mae,mse)

0.8117808263851123 4161.57572494186 25336615.634087294
